In [ ]:
!pip install google-genai

In [2]:
# ENTER YOUR GEMINI_API_KEY HERE

GEMINI_API_KEY = ''

In [5]:
import json
from google import genai

In [ ]:
with open('./data/founders.json', 'r') as file:
    founders = json.load(file)

with open('./data/investors.json', 'r') as file:
    investors = json.load(file)


[{'investor_id': 101, 'name': 'John Doe Ventures', 'preferred_industry': ['FinTech', 'AI/ML'], 'investment_range': [100000, 1000000], 'preferred_stage': ['Seed', 'Series A']}, {'investor_id': 102, 'name': 'Future Health Capital', 'preferred_industry': ['HealthTech'], 'investment_range': [500000, 5000000], 'preferred_stage': ['Series A', 'Series B']}, {'investor_id': 103, 'name': 'AI First Fund', 'preferred_industry': ['AI/ML'], 'investment_range': [250000, 2000000], 'preferred_stage': ['Pre-Seed', 'Seed']}]


In [ ]:
def get_gemini_match_score(founder, investor):
    """Calls Gemini API to analyze founder-investor compatibility"""
    prompt = f"""
    Compare the following startup founder and investor for compatibility:
    Founder: {founder}
    Investor: {investor}
    Provide a match score (0-100) based on industry, funding needs, and startup stage.
    Respond only with the final score.
    """
    client = genai.Client(api_key=GEMINI_API_KEY)

    response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
    )   
    return response.candidates[0].content.parts[0].text.strip()

In [32]:
def get_investors_for_founder(founder):
    results = []
    for investor in investors:
        score = get_gemini_match_score(founder, investor)
        investor['score'] = score
        results.append(investor)
    results = sorted(results, key=lambda d: d['score'], reverse=True)
    results = {'founder': founder, 'matches': results}
    return results


In [33]:
get_investors_for_founder(founders[0])

{'founder': {'founder_id': 1,
  'name': 'Alice Johnson',
  'industry': 'FinTech',
  'startup_stage': 'Seed',
  'funding_required': 500000,
  'traction': '10,000 users, $50K revenue',
  'business_model': 'Subscription-based'},
 'matches': [{'investor_id': 101,
   'name': 'John Doe Ventures',
   'preferred_industry': ['FinTech', 'AI/ML'],
   'investment_range': [100000, 1000000],
   'preferred_stage': ['Seed', 'Series A'],
   'score': '88'},
  {'investor_id': 103,
   'name': 'AI First Fund',
   'preferred_industry': ['AI/ML'],
   'investment_range': [250000, 2000000],
   'preferred_stage': ['Pre-Seed', 'Seed'],
   'score': '50'},
  {'investor_id': 102,
   'name': 'Future Health Capital',
   'preferred_industry': ['HealthTech'],
   'investment_range': [500000, 5000000],
   'preferred_stage': ['Series A', 'Series B'],
   'score': '10'}]}